# Payload Transformer


## Serving

Now that we have trained and saved our model, the next step will be to serve it using `mlserver`. 
For that, we will need to create 2 configuration files: 

- `settings.json`: holds the configuration of our server (e.g. ports, log level, etc.).
- `model-settings.json`: holds the configuration of our model (e.g. input type, runtime to use, etc.).

In [5]:
%%writefile settings.json
{
    "debug": "true"
}

Overwriting settings.json


### `model-settings.json`

In [11]:
%%writefile model-settings.json
{
    "name": "payload-transformer",
    "implementation": "mlserver_payloadtransformer.PayloadTransformerRuntime",
    "parameters": {}
}

Overwriting model-settings.json


### Run Elasticsearch


In [8]:
!docker run -d -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.14.0

0bc45b34663ae80291a75e95fcf8734c6b0665deb008e532fb23a7438d934da5


### Start serving our model

Now that we have our config in-place, we can start the server by running `mlserver start .`. This needs to either be ran from the same directory where our config files are or pointing to the folder where they are.

```shell
mlserver start .
```

Since this command will start the server and block the terminal, waiting for requests, this will need to be ran in the background on a separate terminal.

### Send test inference request

We now have our model being served by `mlserver`.
To make sure that everything is working as expected, let's send a request from our test set.

For that, we can use the Python types that `mlserver` provides out of box, or we can build our request manually.

In [31]:
import requests

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": [1,4],
          "datatype": "FP32",
          "data": [[1,2,3,4]]
        }
    ]
}

endpoint = "http://localhost:8080/"
response = requests.post(endpoint, json=inference_request)

response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

As we can see above, the model predicted the input as the number `8`, which matches what's on the test set.

In [ ]:
y_test[0]